In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sklearn
import sklearn.model_selection
from sklearn import metrics, preprocessing
import pickle
import math
import tensorflow as tf
from os import path, getcwd, chdir

In [3]:
# Create dataframes from our csv files and set indeces

df = pd.read_csv('data/train.csv')
df.set_index('PassengerId', inplace=True)

testdf = pd.read_csv('data/test.csv')
PassengerId = testdf['PassengerId']
testdf.set_index('PassengerId', inplace=True)

data = [df, testdf]

In [4]:
# Preprocess the data by converting non numerical features into numerical categorical features 
# and applying mean imputation to deal with NaN values

for dataframe in data:
    le = preprocessing.LabelEncoder()
    dataframe["Sex"] = le.fit_transform(list(dataframe["Sex"]))
    dataframe["Cabin"] = le.fit_transform(list(dataframe["Cabin"]))
    dataframe["Embarked"] = le.fit_transform(list(dataframe["Embarked"]))
    dataframe.fillna(dataframe.mean(), inplace=True)
    
print(df.head)

<bound method NDFrame.head of              Survived  Pclass  \
PassengerId                     
1                   0       3   
2                   1       1   
3                   1       3   
4                   1       1   
5                   0       3   
...               ...     ...   
887                 0       2   
888                 1       1   
889                 0       3   
890                 1       1   
891                 0       3   

                                                          Name  Sex  \
PassengerId                                                           
1                                      Braund, Mr. Owen Harris    1   
2            Cumings, Mrs. John Bradley (Florence Briggs Th...    0   
3                                       Heikkinen, Miss. Laina    0   
4                 Futrelle, Mrs. Jacques Heath (Lily May Peel)    0   
5                                     Allen, Mr. William Henry    1   
...                                        

In [5]:
# Create our input matrix, label vector, and test input matrix

X = df.drop(['Name', 'Survived', 'Ticket'], axis=1)
y = df['Survived']
X_test = testdf.drop(['Name', 'Ticket'], axis=1)
print(X.head)

<bound method NDFrame.head of              Pclass  Sex        Age  SibSp  Parch     Fare  Cabin  Embarked
PassengerId                                                                
1                 3    1  22.000000      1      0   7.2500    147         2
2                 1    0  38.000000      1      0  71.2833     81         0
3                 3    0  26.000000      0      0   7.9250    147         2
4                 1    0  35.000000      1      0  53.1000     55         2
5                 3    1  35.000000      0      0   8.0500    147         2
...             ...  ...        ...    ...    ...      ...    ...       ...
887               2    1  27.000000      0      0  13.0000    147         2
888               1    0  19.000000      0      0  30.0000     30         2
889               3    0  29.699118      1      2  23.4500    147         2
890               1    1  26.000000      0      0  30.0000     60         0
891               3    1  32.000000      0      0   7.7500

In [6]:
# Normalize the data

X=(X-X.mean())/X.std()
X_test=(X_test-X_test.mean())/X_test.std()

In [7]:
# Setup our data to be fed into our model

dataset = tf.data.Dataset.from_tensor_slices((X.values, y.values))

In [10]:
# Setup our model

model = tf.keras.models.Sequential([
    # Flatten out our input
    tf.keras.layers.Flatten(),
    
    # Setup our first layer
    tf.keras.layers.Dense(1024, activation=tf.nn.relu),
    
    # Setup output layer
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

# Compile our model
model.compile(optimizer='adam', 
              loss = 'sparse_categorical_crossentropy', 
             metrics=['accuracy'])

# Fit model
history = model.fit(
    dataset, 
    epochs=10,
    shuffle=True,
)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/10
      1/Unknown - 0s 186ms/step

ValueError: Shape mismatch: The shape of labels (received (1,)) should equal the shape of logits except for the last dimension (received (8, 2)).

In [100]:
test_pred = model.predict(X_test.values) # Note that we need to feed our model the values or our dataframe X_test
predictions = np.c_[PassengerId, np.argmax(test_pred, axis=1)]
submission = pd.DataFrame(predictions, columns = ['PassengerId', 'Survived'])
print(submission.head)
submission.to_csv("submissions/NNSubmission.csv", index=False)

<bound method NDFrame.head of      PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]>
